In [1]:
import cv2 as cv
print("Current version of openCV=", cv.__version__)
import numpy as np
import os
import glob

Current version of openCV= 4.1.1


In [ ]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
points_per_row = 7
points_per_column = 7
objp = np.zeros((points_per_column*points_per_row,3), np.float32)
objp[:,:2] = np.mgrid[0:points_per_row,0:points_per_column].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('./pinhole_images/*.png')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (points_per_row,points_per_column), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (points_per_row,points_per_column), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print("ret: ", ret)
print("K matrix: ", mtx)
print("D distortions: ", dist)
#print("rotation vecs: ", rvecs)
print("translation vecs: ", tvecs)


In [ ]:
#img_dir = 'pinhole_images/'
img_dir = 'fisheye_images/'
img_name  = '1.png'
dst_dir = 'calibration_results/'
img = cv.imread(img_dir + img_name)
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite(dst_dir+img_dir+img_name, dst)

## Fisheye camera calibration

In [ ]:
# termination criteria
subpix_criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
calibration_flags = (cv.fisheye.CALIB_RECOMPUTE_EXTRINSIC,cv.fisheye.CALIB_CHECK_COND+
                    cv.fisheye.CALIB_FIX_SKEW)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
points_per_row = 7
points_per_column = 7

objp = np.zeros((points_per_column*points_per_row,3), np.float32)
objp[:,:2] = np.mgrid[0:points_per_row,0:points_per_column].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('./fisheye_images/*.png')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, 
                                            (points_per_row,points_per_column), 
                                            cv.CALIB_CB_ADAPTIVE_THRESH+cv.CALIB_CB_FAST_CHECK+cv.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), subpix_criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (points_per_row,points_per_column), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)

In [ ]:
N_OK = len(objpoints)
objpoints=np.array(objpoints, dtype=np.float64)
objpoints = np.reshape(objpoints, (N_OK, 1, points_per_row*points_per_column, 3))
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
cv.destroyAllWindows()

In [ ]:
print("K before: ", K)
calibration_flags = cv.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv.fisheye.CALIB_CHECK_COND+cv.fisheye.CALIB_FIX_SKEW
rms, _, _, _, _ = \
cv.fisheye.calibrate(objpoints,
                        imgpoints, 
                        gray.shape[::-1], 
                        K, 
                        D, 
                        rvecs, 
                        tvecs,
                        calibration_flags,
                        (cv.TERM_CRITERIA_EPS+cv.TERM_CRITERIA_MAX_ITER, 30, 1e-6))
print("Found " + str(N_OK) + " valid images for calibration")
print("K=np.array(" + str(K.tolist()) + ")")

print("D=np.array(" + str(D.tolist()) + ")")

In [ ]:
img = cv.imread('./fisheye_images/1.png')
h,  w = img.shape[:2]
print(h, w)
#newcameramtx, roi = cv.getOptimalNewCameraMatrix(K, D, (h, w), 0, (h, w))
#print('optimal new camera matris: ', newcameramtx, roi)
mapx, mapy = cv.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, (w,h), cv.CV_16SC2)
# undistort
dst = cv.remap(img, map1, map2, 
                   interpolation=cv.INTER_LINEAR,
                  borderMode = cv.BORDER_CONSTANT)
# crop the image
#x, y, w, h = roi
#print("roi: ", roi)
#dst = dst[y:y+h, x:x+w]
cv.imshow("undistorted fisheye: ", dst)
cv.waitKey(500)
#cv.imwrite(dst_dir+img_dir+img_name, dst)


In [ ]:
img = cv.imread('./fisheye_images/1.png')
h,w = img.shape[:2]    
map1, map2 = cv.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv.CV_16SC2)
undistorted_img = cv.remap(img, map1, map2, interpolation=cv.INTER_LINEAR, borderMode=cv.BORDER_CONSTANT)
cv.imshow("undistorted", undistorted_img)
cv.waitKey(500)
#cv.destroyAllWindows()

In [ ]:
print("K: ", K)
print("D: ", D)
print("eye: ", np.eye(3))
data_dir = './fisheye_images/'
filename = "{}.png"
dst_dir = './calibration_results/fisheye_images/'
for img_id in range(len(images)):
    img_name = data_dir + filename.format(img_id)
    img = cv.imread(img_name)
    map1, map2 = cv.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv.CV_16SC2)
    undistorted_img = cv.remap(img, map1, map2, interpolation=cv.INTER_LINEAR, borderMode=cv.BORDER_CONSTANT)
    cv.imshow('img_undistorted', undistorted_img)
    cv.waitKey(500)
    #cv.imwrite(dst_dir + filename.format(img_id), undistorted_img)
cv.destroyAllWindows()

In [ ]:
#Fisheye calibration

N_OK = len(objpoints)
objpointsL=np.array(objpoints, dtype=np.float64)
objpointsL = np.reshape(objpointsL, (N_OK, 1, points_per_row*points_per_column, 3))
KL = np.zeros((3, 3))
DL = np.zeros((4, 1))
rvecsL = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecsL = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
calibration_flags = cv.fisheye.CALIB_RECOMPUTE_EXTRINSIC
rms, _, _, _, _ = \
cv.fisheye.calibrate(objpointsL,
                        imgpointsL, 
                        grayL.shape[::-1], 
                        KL, 
                        DL, 
                        rvecsL, 
                        tvecsL,
                        calibration_flags,
                        (cv.TERM_CRITERIA_EPS+cv.TERM_CRITERIA_MAX_ITER, 30, 1e-6))
print("Found " + str(N_OK) + " valid images for calibration")
print("K=np.array(" + str(KL.tolist()) + ")")
print("D=np.array(" + str(DL.tolist()) + ")")


## Stereo

In [7]:
left_calibration_path = './calibration_results/fisheye_images/{}.png'
right_calibration_path = './calibration_results/pinhole_images/{}.png'

In [8]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
points_per_row = 7
points_per_column = 7
objp = np.zeros((points_per_column*points_per_row,3), np.float32)
objp[:,:2] = np.mgrid[0:points_per_row,0:points_per_column].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in Right image plane.
imgpointsR = [] # 2d points in Left image plane.
imagesR = glob.glob('./calibration_results/pinhole_images/*.png')
imagesL = glob.glob('./calibration_results/fisheye_images/*.png')
for img_id in range(len(imagesR)):
    fnameL = left_calibration_path.format(img_id)
    fnameR = right_calibration_path.format(img_id)
    imgL = cv.imread(fnameL)
    imgR = cv.imread(fnameR)
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    retL, cornersL = cv.findChessboardCorners(grayL, (points_per_row,points_per_column), None)
    retR, cornersR = cv.findChessboardCorners(grayR, (points_per_row,points_per_column), None)
    # If found on both, add object points, image points (after refining them)
    if retL == True and retR == True:
        objpoints.append(objp)
        corners2L = cv.cornerSubPix(grayL,cornersL, (11,11), (-1,-1), criteria)
        corners2R = cv.cornerSubPix(grayR,cornersR, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)
        imgpointsR.append(cornersR)
        # Draw and display the corners
        cv.drawChessboardCorners(imgL, (points_per_row,points_per_column), corners2L, retL)
        cv.drawChessboardCorners(imgR, (points_per_row,points_per_column), corners2R, retR)
        cv.imshow('img', imgL)
        cv.waitKey(500)
        cv.imshow('img', imgR)
        cv.waitKey(500)
cv.destroyAllWindows()

## Store undistorted images

In [ ]:
#pinhole Right camera calibration
#print(objpoints)
ret, KR, DR, rvecsR, tvecsR = cv.calibrateCamera(objpoints, imgpointsR, grayR.shape[::-1], None, None)
print("Right camera calibration\n")
print("ret: ", ret)
print("K matrix: ", KR)
print("D distortions: ", DR)
#print("rotation vecs: ", rvecsR)
print("translation vecs: ", tvecsR)
#pinhole Left camera calibration
#print(objpoints)
ret, KL, DL, rvecsL, tvecsL = cv.calibrateCamera(objpoints, imgpointsL, grayL.shape[::-1], None, None)
print("Left camera calibration\n")
print("ret: ", ret)
print("K matrix: ", KL)
print("D distortions: ", DL)
#print("rotation vecs: ", rvecsR)
print("translation vecs: ", tvecsL)

In [ ]:
#img_dir = 'pinhole_images/'
img_dirL = 'fisheye_images/'
img_dirR = "pinhole_images/"
img_name  = '{}.png'
dst_dir = 'calibration_results/'
#get image size
img = cv.imread(left_calibration_path.format(img_id))
img_size = img.shape[:2]
h,  w = img.shape[:2]
width, height = img_size
for img_id in range(len(imagesR)):
    imgL = cv.imread(img_dirL + img_name.format(img_id))
    imgR = cv.imread(img_dirR + img_name.format(img_id))
    newcameramtxL, roiL = cv.getOptimalNewCameraMatrix(KL, DL, (w,h), 0, (w,h))
    newcameramtxR, roiR = cv.getOptimalNewCameraMatrix(KR, DR, (w,h), 0, (w,h))
    # undistort
    dstL = cv.undistort(imgL, KL, DL, None, newcameramtxL)
    dstR = cv.undistort(imgR, KR, DR, None, newcameramtxR)
    # crop the images
    #crop right
    x, y, w, h = roiL
    dstL = dstL[y:y+h, x:x+w]
    cv.imwrite(dst_dir+img_dirL+img_name.format(img_id), dstL)
    #crop left
    x, y, w, h = roiR
    dstR = dstR[y:y+h, x:x+w]
    cv.imwrite(dst_dir+img_dirR+img_name.format(img_id), dstR)

In [ ]:
cv.destroyAllWindows()

## Perform image rectification

In [21]:
img = cv.imread(left_calibration_path.format(img_id))
width, height = img.shape[:-1]
R = None
T = None
E = None
F = None
stereocalib_criteria = (cv.TERM_CRITERIA_MAX_ITER + cv.TERM_CRITERIA_EPS, 100, 1e-5)
stereocalib_flags = cv.CALIB_FIX_ASPECT_RATIO | cv.CALIB_ZERO_TANGENT_DIST | cv.CALIB_SAME_FOCAL_LENGTH | cv.CALIB_RATIONAL_MODEL | cv.CALIB_FIX_K3 | cv.CALIB_FIX_K4 | cv.CALIB_FIX_K5
print(width, height)
retval, KL, DL, KR, DR, R, T, E, F = \
 cv.stereoCalibrate(objpoints, imgpointsL, imgpointsR,
                                #imageSize = (width, height),
                                imageSize = (height, width),
                                cameraMatrix1 = None,
                                distCoeffs1 = None,
                                cameraMatrix2 = None,
                                distCoeffs2 = None,
                                R=R,
                                T=T,
                                E=E,
                                F=F,
                                #flags=cv.CALIB_FIX_INTRINSIC
                                flags=stereocalib_flags,
                                criteria=stereocalib_criteria
                   )

720 1280


In [10]:
print(retval)
print("cameraM1: \n", KL)
print("distCoeffs1: \n", DL)
print("cameraM2: \n", KR)
print("distCoeffs2: \n", DR)
print("R: ", R)
print("T: ", T)
print("E: ", E)
print("F: ", F)

1.678273585664323
cameraM1: 
 [[1.09483213e+03 0.00000000e+00 6.24961734e+02]
 [0.00000000e+00 1.09483213e+03 3.36673363e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distCoeffs1: 
 [[  0.01618444   2.55991502   0.           0.           0.
    0.           0.         -13.73483473   0.           0.
    0.           0.           0.           0.        ]]
cameraM2: 
 [[1.09483213e+03 0.00000000e+00 6.16310042e+02]
 [0.00000000e+00 1.09483213e+03 3.62861569e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distCoeffs2: 
 [[-0.00178041  0.10217565  0.          0.          0.          0.
   0.          0.23813067  0.          0.          0.          0.
   0.          0.        ]]
R:  [[ 9.99539762e-01  3.26388523e-04  3.03340853e-02]
 [ 1.39842397e-03  9.98383350e-01 -5.68219287e-02]
 [-3.03035917e-02  5.68381970e-02  9.97923400e-01]]
T:  [[  3.85291541]
 [  1.29027583]
 [-10.04987305]]
E:  [[-0.02504601 10.10696288  0.71654327]
 [-9.92849055 -0.22227293 -4.14976815]
 [-1.284293

In [11]:
img_id = 1

imgL = cv.imread('./calibration_results/fisheye_images/{}.png'.format(img_id))
imgR = cv.imread('./calibration_results/pinhole_images/{}.png'.format(img_id))
img_size = imgL.shape[:2]
alpha = 0 ### =0 crop, =1 keep dimensions
print(img_size)

RL = np.zeros(shape=(3,3))
RR = np.zeros(shape=(3,3))
PL = np.zeros(shape=(3,4))
PR = np.zeros(shape=(3,4))

"""out = cv.stereoRectify(KL, DL, KR, DR, (width, height), R=R, T=T, 
                       R1=RL, R2=RR, P1=PL, P2=PR,
                      Q=None, flags=cv.CALIB_ZERO_DISPARITY, alpha=alpha, newImageSize=(0,0))
"""
RL, RR, PL, PR, Q, roiL, roiR = cv.stereoRectify(KL, DL, KR, DR, (width, height), R=R, T=T, alpha=alpha)


(720, 1280)


In [12]:
print("RL: \n", RL)
print("RR: \n", RR)
print("PL: \n", PL)
print("RR: \n", PR)
print("Q: ", Q)
print("roiL: \n", roiL)
print("roiR: \n", roiR)


RL: 
 [[ 0.38352591  0.06625629 -0.92115035]
 [-0.09305787  0.99511925  0.03283156]
 [ 0.91882975  0.07312853  0.38781969]]
RR: 
 [[ 0.35542877  0.11902705 -0.92709382]
 [-0.09169433  0.9915148   0.09214416]
 [ 0.93019489  0.05225856  0.363327  ]]
PL: 
 [[ 3.37277318e+02  0.00000000e+00 -2.57978958e+03  0.00000000e+00]
 [ 0.00000000e+00  3.37277318e+02  1.99137965e+03  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]
RR: 
 [[ 3.37277318e+02  0.00000000e+00 -2.57978958e+03  3.65615017e+03]
 [ 0.00000000e+00  3.37277318e+02  1.99137965e+03  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]
Q:  [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  2.57978958e+03]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -1.99137965e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.37277318e+02]
 [ 0.00000000e+00  0.00000000e+00 -9.22493064e-02  0.00000000e+00]]
roiL: 
 (0, 0, 0, 0)
roiR: 
 (0, 0, 0, 0)


In [13]:
print("Applying Undistort\n")
map1x, map1y = cv.initUndistortRectifyMap(KL, DL, RL, PL, (width, height), cv.CV_32FC1)
map2x, map2y = cv.initUndistortRectifyMap(KR, DR, RR, PR, (width, height), cv.CV_32FC1)

print("Undistort complete\n")

"""
for img_id in range(len(imagesR)):
    fnameL = left_calibration_path.format(img_id)
    fnameR = right_calibration_path.format(img_id)
    print("fname: ", fnameL, fnameR)
    imgL = cv.imread(fnameL)
    imgR = cv.imread(fnameR)
    imgU1 = np.zeros((height,width,3), np.uint8)
    imgU1 = cv.remap(imgL, map1x, map1y, cv.INTER_LINEAR, imgU1, cv.BORDER_CONSTANT, 0)
    imgU2 = cv.remap(imgR, map2x, map2y, cv.INTER_LINEAR)
    cv.imshow("imageL", imgL);
    cv.imshow("imageR", imgR);
    cv.imshow("image1L", imgU1);
    cv.imshow("image2R", imgU2);
    k = cv.waitKey(5);
    if(k==27):
        break;
"""
print("Maps1: \n", map1x, map1y)
print("Maps2: \n", map2x, map2y)

#image remap example
img_id = 1
fnameL = left_calibration_path.format(img_id)
fnameR = right_calibration_path.format(img_id)
print("fname: ", fnameL, fnameR)
imgL = cv.imread(fnameL)
imgR = cv.imread(fnameR)
cv.imshow('imgL', imgL)
cv.waitKey(500)

left_img_remap = cv.remap(imgL, map1x, map1y, cv.INTER_LANCZOS4)
right_img_remap = cv.remap(imgR, map2x, map2y, cv.INTER_LANCZOS4)

Applying Undistort

Undistort complete

Maps1: 
 [[   818.9658     819.15674    819.34784 ...   1031.7202    1032.1698
    1032.62   ]
 [   819.19257    819.3838     819.57526 ...   1032.3688    1032.8195
    1033.2706 ]
 [   819.4198     819.6113     819.8031  ...   1033.019     1033.4707
    1033.923  ]
 ...
 [-35808.555   -34417.484   -33130.113   ...  -1093.2662   -1091.6613
   -1090.0607 ]
 [-33609.77    -32380.574   -31237.816   ...  -1090.483    -1088.8854
   -1087.2921 ]
 [-31664.791   -30570.734   -29549.486   ...  -1087.7141   -1086.1238
   -1084.5377 ]] [[  103.27883    103.11817    102.95736  ...   -65.217896   -65.559074
    -65.90068 ]
 [  103.12157    102.96069    102.799675 ...   -65.64953    -65.99139
    -66.333664]
 [  102.964035   102.802956   102.64173  ...   -66.08217    -66.4247
    -66.76765 ]
 ...
 [14082.679    13552.432    13061.708    ...   826.5485     825.9072
    825.26764 ]
 [13229.033    12761.045    12325.962    ...   824.74225    824.10455
    823.468

In [15]:
cv.imshow("imgL remap", left_img_remap)
cv.imshow("imgR remap", right_img_remap)
cv.waitKey(500)
cv.destroyAllWindows()

In [ ]:
print("Applying Undistort\n")
#map1x, map1y = cv.initUndistortRectifyMap(KL, DL, R1, P1, (width, height), cv.CV_32FC1)
#map2x, map2y = cv.initUndistortRectifyMap(KR, DR, R2, P2, (width, height), cv.CV_32FC1)
map1x, map1y = cv.initUndistortRectifyMap(KL, DL, RL, PL, (width, height), cv.CV_16SC2)
map2x, map2y = cv.initUndistortRectifyMap(KR, DR, RR, PR, (width, height), cv.CV_16SC2)

print("Undistort complete\n")

for img_id in range(len(imagesR)):
    fnameL = left_calibration_path.format(img_id)
    fnameR = right_calibration_path.format(img_id)
    print("fname: ", fnameL, fnameR)
    imgL = cv.imread(fnameL)
    imgR = cv.imread(fnameR)
    #imgU1 = np.zeros((height,width,3), np.uint8)
    #imgU1 = cv.remap(imgL, map1x, map1y, cv.INTER_LINEAR)
    #imgU2 = cv.remap(imgR, map2x, map2y, cv.INTER_LINEAR)
    imgU1 = cv.remap(imgL, map1x, map1y, cv.INTER_LANCZOS4)
    imgU2 = cv.remap(imgR, map2x, map2y, cv.INTER_LANCZOS4)
    cv.imshow("imageL", imgL);
    cv.imshow("imageR", imgR);
    cv.imshow("imageL", imgU1);
    cv.imshow("imageR", imgU2);
    k = cv.waitKey(500);
    if(k==27):
        break;

In [ ]:
print("imageL shape: ", imgL.shape)
print("imageR shape: ", imgR.shape)
cv.destroyAllWindows()